In [1]:
# orb brute force works
import numpy as np
import cv2

MIN_MATCH_COUNT = 10

cap = cv2.VideoCapture('../../vid/swarm_drone.mp4')
img = cv2.imread('../../img/captures/img2.jpg')
orb = cv2.ORB_create(nfeatures=np.int(1e2))
#orb = cv2.ORB_create()

scale = 1
img = cv2.resize(img, (0,0), fx=1 / scale, fy=1 / scale) 

kp_prev, des_prev = orb.detectAndCompute(img,None)
crop = cv2.drawKeypoints(img, kp_prev, None, color=(0,255,0), flags=0)
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

while True:
    ret, frame = cap.read()
    
    frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale) 
    if ret == True:
        kp_frame, des_frame = orb.detectAndCompute(frame, None)
        
        
        # create BFMatcher object
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        # Match descriptors.
        matches = bf.match(des_prev,des_frame)
        # Sort them in the order of their distance.
        matches = sorted(matches, key = lambda x:x.distance)
        # Draw first 10 matches.
        frame = cv2.drawKeypoints(frame, kp_frame, None, color=(0,255,0), flags=0)
        out = cv2.drawMatches(crop,kp_prev,frame,kp_frame,matches[:10], None, flags=2)
        cv2.imshow('out', out)
        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()

In [4]:
# orb brute force works
import numpy as np
import cv2
cap = cv2.VideoCapture('../../vid/swarm_drone.mp4')
orb = cv2.ORB_create(nfeatures=1000)
#orb = cv2.ORB_create()
#cap.set(cv2.CAP_PROP_POS_FRAMES, 1000)

whole = 100 // scale
y, x = 230 // scale, 550 // scale
#y, x = 430 // scale, 850 // scale
img = cv2.imread('../../img/captures/img2.jpg')
scale = 1
img = cv2.resize(img, (0,0), fx=1 / scale, fy=1 / scale) 

kp_prev, des_prev = orb.detectAndCompute(img,None)
crop = cv2.drawKeypoints(img, kp_prev, None, color=(0,255,0), flags=0)
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()



while True:
    ret, frame = cap.read()
    
    
    if ret == True:
        frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale) 
        height,width,depth = frame.shape
        mask = np.zeros((height,width), np.uint8)
        cv2.rectangle(mask, (x,y), (x+whole,y+whole),256,thickness=-1)
        masked = cv2.bitwise_and(frame, frame, mask=mask)
        kp, des = orb.detectAndCompute(masked,None)
        # create BFMatcher object
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        # Match descriptors.
        matches = bf.match(des_prev,des)
        # Sort them in the order of their distance.
        matches = sorted(matches, key = lambda x:x.distance)
        # Draw first 10 matches.
        out = cv2.drawMatches(crop,kp_prev,masked,kp,matches[:10], None, flags=2)
        cv2.imshow('out', out)

        size = len(matches)
        if size > 5:
            if size > 10:
                size = 10
            arr = np.zeros((2,size))
            for i, m in enumerate(matches[:size]):
                x, y = kp[m.trainIdx].pt
                arr[0, i] = x
                arr[1, i] = y
            mean_x, mean_y = np.int(np.mean(arr[0])), np.int(np.mean(arr[1]))
            x = mean_x - whole//2
            y = mean_y - whole//2
        cv2.rectangle(frame, (x, y), (x+whole, y+whole), (0,255,0), 1)
        cv2.imshow('frame', frame)
        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()